# Using the Long Readout Time (LRT) option

This notebook demonstrates how to make use of the long readout time option on SHFQA and SHFQC instruments. 

## Python Imports

Import the necessary DSL classes from LabOne Q

In [1]:
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)
from laboneq.simple import *

## Device Setup

Create the device setup instance including qubits.

In [25]:
# specify the number of qubits you want to use
number_of_qubits = 2

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10056"}],
    shfsg=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 8,
            "options": "SHFSG8/RTR",
        }
    ],
    shfqa=[
        {
            "serial": "DEV12002",
            "zsync": 2,
            "number_of_channels": 4,
            "readout_multiplex": 6,
            "options": "SHFQA4/LRT",
        }
    ],
    include_flux_lines=False,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_tuneable_qubit_setup",
)

q0, q1 = qubits[:2]

In [ ]:
use_emulation = True

session = Session(device_setup)
session.connect(do_emulation=use_emulation, ignore_version_mismatch=True)

## Experiment

Define an experiment that makes use of the long readout time functionality.

* Specify a readout pulse or integration kernel that is longer than 4096 samples or 2.043 us.
* The readout pulse needs to be compressible, meaning that the middle part of it needs to be constant. Note that the total length of the non-constant part still cannot exceed 4096 samples. 
* Enable hardware modulation for both the readout and acquisition logical signal.

In [27]:
def long_readout_experiment(qubit, readout_time=10e-6, average_count=2**3):
    exp = Experiment(
        signals=[
            ExperimentSignal("drive", map_to=qubit.signals["drive"]),
            ExperimentSignal("acquire", map_to=qubit.signals["acquire"]),
            ExperimentSignal("measure", map_to=qubit.signals["measure"]),
        ]
    )
    # generate readout pulse - enable compression to use pulses longer than 2us
    readout_pulse = pulse_library.const(
        length=readout_time,
        amplitude=0.5,
        can_compress=True,
    )

    # generate and set the configuration for the long readout - use hardware modulation for long readout
    experiment_configuration = Calibration()
    experiment_configuration["measure"] = SignalCalibration(
        oscillator=Oscillator(
            frequency=qubit.parameters.readout_frequency,
            modulation_type=ModulationType.HARDWARE,
        )
    )
    experiment_configuration["acquire"] = SignalCalibration(
        oscillator=Oscillator(
            frequency=qubit.parameters.readout_frequency,
            modulation_type=ModulationType.HARDWARE,
        )
    )
    exp.set_calibration(experiment_configuration)

    with exp.acquire_loop_rt(
        count=average_count,
        acquisition_type=AcquisitionType.INTEGRATION,
        averaging_mode=AveragingMode.SINGLE_SHOT,
    ):
        with exp.section(uid="drive"):
            exp.play("drive", pulse=pulse_library.drag())
        with exp.section(uid="long_readout", play_after="drive"):
            exp.play("measure", pulse=readout_pulse)
            exp.acquire("acquire", kernel=readout_pulse, handle="my_results")
            exp.delay("acquire", 500e-9)

    return exp

In [28]:
# generate the experiment with a 20us long readout pulse
my_exp = long_readout_experiment(qubit=q0, readout_time=30e-6)

In [ ]:
compiled_exp = session.compile(my_exp)

In [ ]:
my_results = session.run(compiled_exp)

In [ ]:
# inspect the pulse sheet

# show_pulse_sheet(name="long readout experiment", compiled_experiment=compiled_exp)